In [115]:
#Importing Libraries
import torch
import torch.nn as nn

In [116]:
!nvidia-smi

Thu Feb  2 15:50:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.92.01    Driver Version: 528.02       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   48C    P8    N/A /  55W |    520MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [117]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
device

'cuda'

In [118]:
# Creating a dataset
weight = 0.3
bias = 0.9
X = torch.arange(0,1,0.02).unsqueeze(dim=1)
y = weight * X + bias
X.shape, y.shape

(torch.Size([50, 1]), torch.Size([50, 1]))

In [120]:
#Train-test split
train_split = int(0.8*len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]
len(X_train), len(y_train), len(X_test), len(y_test)

(40, 40, 10, 10)

In [122]:
class Linear_Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.weights = nn.Parameter(torch.randn(1, requires_grad=True, dtype = torch.float))
        self.bias = nn.Parameter(torch.randn(1, requires_grad=True, dtype = torch.float))
    
    def forward(self, x):
        return self.weights * x + self.bias

In [123]:
model = Linear_Model()
model.state_dict()

OrderedDict([('weights', tensor([0.7649])), ('bias', tensor([0.8263]))])

We can use `nn.Linear` to calculate weights and bias for us. `nn.Linear` is kind of a wrapper for what we defined in `Linear_Model`.

In [124]:
class Linear(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.linear_layer = nn.Linear(in_features=1, out_features=1) #size of ouput
    
    def forward(self, X):
        return self.linear_layer(X)

In [125]:
Model_2 = Linear()
Model_2.state_dict()

OrderedDict([('linear_layer.weight', tensor([[-0.7474]])),
             ('linear_layer.bias', tensor([0.9097]))])

In [126]:
next(model.parameters()).device

device(type='cpu')

Change the device to gpu if available

In [127]:
model.to(device)
next(model.parameters()).device

device(type='cuda', index=0)

In [128]:
# Loss and Optimizer
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01)

In [142]:
#Training
epochs = 100

#change the device of tensors to new device as model device is changed

X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device) 

for epoch in range(epochs):
    # When passed a value to model, it starts forward() method
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    # Backpropogate to find graidents
    loss.backward()
    # Using learning rate try to reach minima
    optimizer.step()
    # Zero the calculated gradients beacuse there should be calculated fresh in next epoch
    optimizer.zero_grad()
    

In [145]:
model.state_dict()

OrderedDict([('weights', tensor([0.3863], device='cuda:0')),
             ('bias', tensor([0.8638], device='cuda:0'))])

Weights and bias are almost optimized to the original weights and bias we gave when creating the datatset.

In [144]:
#Testing
with torch.inference_mode(): #Doesn't track derviates and doesn't optimize
    y_test_pred = model(X_test)
    test_loss = loss_fn(y_test_pred, y_test)
loss, test_loss

(tensor(0.0177, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.0405, device='cuda:0'))

#Saving
# https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-loading-model-for-inference